# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 7 2023 8:42AM,256797,10,0086359661,ISDIN Corporation,Released
1,Feb 7 2023 8:42AM,256797,10,0086359660,ISDIN Corporation,Released
2,Feb 7 2023 8:42AM,256797,10,0086359671,ISDIN Corporation,Released
3,Feb 7 2023 8:42AM,256797,10,0086359669,ISDIN Corporation,Released
4,Feb 7 2023 8:42AM,256797,10,0086359673,ISDIN Corporation,Released
5,Feb 7 2023 8:42AM,256797,10,0086359678,ISDIN Corporation,Released
6,Feb 7 2023 8:42AM,256797,10,0086359679,ISDIN Corporation,Released
7,Feb 7 2023 8:42AM,256797,10,0086359682,ISDIN Corporation,Released
8,Feb 7 2023 8:42AM,256797,10,0086359681,ISDIN Corporation,Released
9,Feb 7 2023 8:42AM,256797,10,0086359692,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,256792,Released,15
32,256793,Released,42
33,256795,Released,1
34,256796,Released,40
35,256797,Released,38


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256792,NaN,NaN,15.0
256793,NaN,NaN,42.0
256795,NaN,NaN,1.0
256796,NaN,NaN,40.0
256797,NaN,NaN,38.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0.0,0.0,2.0
256684,0.0,0.0,1.0
256685,0.0,1.0,0.0
256686,0.0,1.0,0.0
256695,32.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256676,0,0,2
256684,0,0,1
256685,0,1,0
256686,0,1,0
256695,32,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,0,0,2
1,256684,0,0,1
2,256685,0,1,0
3,256686,0,1,0
4,256695,32,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256676,,,2
1,256684,,,1
2,256685,,1,
3,256686,,1,
4,256695,32,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 7 2023 8:42AM,256797,10,ISDIN Corporation
38,Feb 7 2023 8:40AM,256796,10,ISDIN Corporation
78,Feb 7 2023 8:40AM,256793,10,ISDIN Corporation
120,Feb 7 2023 8:33AM,256795,19,"GCH Granules USA, Inc."
121,Feb 7 2023 8:30AM,256761,10,Bio-PRF
128,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation
129,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation
130,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation
136,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation
151,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 7 2023 8:42AM,256797,10,ISDIN Corporation,,,38
1,Feb 7 2023 8:40AM,256796,10,ISDIN Corporation,,,40
2,Feb 7 2023 8:40AM,256793,10,ISDIN Corporation,,,42
3,Feb 7 2023 8:33AM,256795,19,"GCH Granules USA, Inc.",,,1
4,Feb 7 2023 8:30AM,256761,10,Bio-PRF,,,7
5,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,,,1
6,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,,,7
7,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,,,14
8,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,,,1
9,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:42AM,256797,10,ISDIN Corporation,38,,
1,Feb 7 2023 8:40AM,256796,10,ISDIN Corporation,40,,
2,Feb 7 2023 8:40AM,256793,10,ISDIN Corporation,42,,
3,Feb 7 2023 8:33AM,256795,19,"GCH Granules USA, Inc.",1,,
4,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7,,
5,Feb 7 2023 8:30AM,256781,10,ISDIN Corporation,1,,
6,Feb 7 2023 8:30AM,256764,10,ISDIN Corporation,7,,
7,Feb 7 2023 8:30AM,256767,10,ISDIN Corporation,14,,
8,Feb 7 2023 8:30AM,256777,10,ISDIN Corporation,1,,
9,Feb 7 2023 8:30AM,256778,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:42AM,256797,10,ISDIN Corporation,38,,
1,Feb 7 2023 8:40AM,256796,10,ISDIN Corporation,40,,
2,Feb 7 2023 8:40AM,256793,10,ISDIN Corporation,42,,
3,Feb 7 2023 8:33AM,256795,19,"GCH Granules USA, Inc.",1,,
4,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:42AM,256797,10,ISDIN Corporation,38.0,NaN,NaN
1,Feb 7 2023 8:40AM,256796,10,ISDIN Corporation,40.0,NaN,NaN
2,Feb 7 2023 8:40AM,256793,10,ISDIN Corporation,42.0,NaN,NaN
3,Feb 7 2023 8:33AM,256795,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 8:42AM,256797,10,ISDIN Corporation,38.0,0.0,0.0
1,Feb 7 2023 8:40AM,256796,10,ISDIN Corporation,40.0,0.0,0.0
2,Feb 7 2023 8:40AM,256793,10,ISDIN Corporation,42.0,0.0,0.0
3,Feb 7 2023 8:33AM,256795,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Feb 7 2023 8:30AM,256761,10,Bio-PRF,7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3594693,243.0,0.0,2.0
12,256784,14.0,0.0,0.0
15,513430,6.0,0.0,52.0
16,770139,0.0,3.0,0.0
19,1797337,11.0,8.0,11.0
20,770361,6.0,0.0,0.0
21,513369,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3594693,243.0,0.0,2.0
1,12,256784,14.0,0.0,0.0
2,15,513430,6.0,0.0,52.0
3,16,770139,0.0,3.0,0.0
4,19,1797337,11.0,8.0,11.0
5,20,770361,6.0,0.0,0.0
6,21,513369,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,243.0,0.0,2.0
1,12,14.0,0.0,0.0
2,15,6.0,0.0,52.0
3,16,0.0,3.0,0.0
4,19,11.0,8.0,11.0
5,20,6.0,0.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,243.0
1,12,Released,14.0
2,15,Released,6.0
3,16,Released,0.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,2.0,0.0,52.0,0.0,11.0,0.0,0.0
Executing,0.0,0.0,0.0,3.0,8.0,0.0,1.0
Released,243.0,14.0,6.0,0.0,11.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,52.0,0.0,11.0,0.0,0.0
1,Executing,0.0,0.0,0.0,3.0,8.0,0.0,1.0
2,Released,243.0,14.0,6.0,0.0,11.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,52.0,0.0,11.0,0.0,0.0
1,Executing,0.0,0.0,0.0,3.0,8.0,0.0,1.0
2,Released,243.0,14.0,6.0,0.0,11.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()